In [2]:
#!/usr/bin/env python3
# vcf_to_csv.py — simple vCard (.vcf) -> CSV converter for contacts
import sys
import csv
from collections import defaultdict

def parse_vcf(path):
    entries = []
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        current = defaultdict(list)
        for line in f:
            line = line.strip()
            if not line: 
                continue
            if line.upper() == 'BEGIN:VCARD':
                current = defaultdict(list)
            elif line.upper() == 'END:VCARD':
                entries.append(current)
            else:
                if ':' in line:
                    parts = line.split(':', 1)
                    key = parts[0].upper()
                    val = parts[1]
                    current[key].append(val)
        return entries

def get_primary(values):
    return values[0] if values else ''

def vcard_to_rows(entries):
    rows = []
    for e in entries:
        row = {
            'Full Name': get_primary(e.get('FN') or e.get('N')),
            'Name Field': get_primary(e.get('N')),
            'Phone': ';'.join(e.get('TEL', [])),
            'Email': ';'.join(e.get('EMAIL', [])),
            'Org': get_primary(e.get('ORG')),
            'Title': get_primary(e.get('TITLE')),
            'Note': get_primary(e.get('NOTE')),
        }
        rows.append(row)
    return rows

def main():
    if len(sys.argv) < 3:
        print("Usage: python vcf_to_csv.py input.vcf output.csv")
        sys.exit(1)
    in_vcf = sys.argv[1]
    out_csv = sys.argv[2]
    entries = parse_vcf(in_vcf)
    rows = vcard_to_rows(entries)
    keys = ['Full Name','Name Field','Phone','Email','Org','Title','Note']
    with open(out_csv, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        for r in rows:
            writer.writerow(r)
    print(f"Wrote {len(rows)} contacts to {out_csv}")

if __name__ == '__main__':
    main()


Usage: python vcf_to_csv.py input.vcf output.csv


SystemExit: 1

# Contacts need to load into CRM
* convert the contacts in to vcf file to csv file and merge them in to different groups